# Prequisites

In [1]:
import torchvision
from torchvision import transforms, datasets 
import torch
import os
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from torchinfo import summary
import numpy as np

**tensorboard --logdir=./log**

* https://github.com/pytorch/kineto/blob/main/tb_plugin/docs/gpu_utilization.md
* https://docs.pytorch.org/tutorials/intermediate/tensorboard_profiler_tutorial.html

# Data

In [2]:
train_data_path = "..//train"

transformations = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485, 0.456, 0.406],
                          std = [0.229, 0.224, 0.225])
])

train_data = torchvision.datasets.ImageFolder(train_data_path, transform=transformations)

val_data_path = "..//val"
val_data = torchvision.datasets.ImageFolder(val_data_path, transform=transformations)

test_data_path = "..//test"
test_data = torchvision.datasets.ImageFolder(test_data_path, transform=transformations)

# Dataloader

batch_size = 512 #Num of images sent to the network once before updating it

train_data_loader = DataLoader(train_data, batch_size=batch_size)
val_data_loader = DataLoader(val_data, batch_size=batch_size)
test_data_loader = DataLoader(test_data, batch_size=batch_size)

num_classes = 151

# VGG 16

In [3]:
model = torchvision.models.vgg16(weights=torchvision.models.VGG16_Weights.DEFAULT)

for layer in model.parameters():
    layer.requires_grad = False

for name, layer in model.named_parameters():
    if "classifier" in name:
        layer.requires_grad = True

in_features = model.classifier[6].in_features 
model.classifier[6] = torch.nn.Linear(in_features, num_classes)
model.classifier[6].requires_grad_ = True

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

model = model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.classifier.parameters(), lr=0.001)

In [4]:
def train(data):
    inputs, labels = data[0].to(device=device), data[1].to(device=device)
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [6]:
prof = torch.profiler.profile(
        schedule=torch.profiler.schedule(wait=1, warmup=1, active=3, repeat=1),
        on_trace_ready=torch.profiler.tensorboard_trace_handler('./log/vgg16'),
        record_shapes=True,
        with_stack=True)
prof.start()
for step, batch_data in enumerate(train_data_loader):
    prof.step()
    if step >= 1 + 1 + 3:
        break
    train(batch_data)
prof.stop()

# AlexNet

In [7]:
model = torchvision.models.alexnet(weights=torchvision.models.AlexNet_Weights.DEFAULT)

for layer in model.parameters():
    layer.requires_grad = False

for name, layer in model.named_parameters():
    if "classifier" in name:
        layer.requires_grad = True

in_features = model.classifier[6].in_features 
model.classifier[6] = torch.nn.Linear(in_features, num_classes)
model.classifier[6].requires_grad_ = True

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

model = model.to(device)

In [8]:
def train(data):
    inputs, labels = data[0].to(device=device), data[1].to(device=device)
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

prof = torch.profiler.profile(
        schedule=torch.profiler.schedule(wait=1, warmup=1, active=3, repeat=1),
        on_trace_ready=torch.profiler.tensorboard_trace_handler('./log/alexnet'),
        record_shapes=True,
        with_stack=True)
prof.start()
for step, batch_data in enumerate(train_data_loader):
    prof.step()
    if step >= 1 + 1 + 3:
        break
    train(batch_data)
prof.stop()

# Mobilenet V3

In [9]:
model = torchvision.models.mobilenet_v3_small(weights=torchvision.models.MobileNet_V3_Small_Weights.DEFAULT)

for layer in model.parameters():
    layer.requires_grad = False

for name, layer in model.named_parameters():
    if "classifier" in name:
        layer.requires_grad = True

in_features = model.classifier[3].in_features 
model.classifier[3] = torch.nn.Linear(in_features, num_classes)
model.classifier[3].requires_grad_ = True

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

model = model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.classifier.parameters(), lr=0.001)

In [10]:
def train(data):
    inputs, labels = data[0].to(device=device), data[1].to(device=device)
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

prof = torch.profiler.profile(
        schedule=torch.profiler.schedule(wait=1, warmup=1, active=3, repeat=1),
        on_trace_ready=torch.profiler.tensorboard_trace_handler('./log/mobilenetv3_small'),
        record_shapes=True,
        with_stack=True)
prof.start()
for step, batch_data in enumerate(train_data_loader):
    prof.step()
    if step >= 1 + 1 + 3:
        break
    train(batch_data)
prof.stop()